# How to use the ifes_apt_tc_data_modeling library

## Load ifes_apt_tc_data_modeling library

In [ ]:
import re
import os
import numpy as np
import h5py
import pandas as pd
from jupyterlab_h5web import H5Web
from ifes_apt_tc_data_modeling.utils.utils import create_isotope_vector, \
    isotope_vector_to_dict_keyword, isotope_vector_to_human_readable_name, \
    isotope_to_hash, hash_to_isotope
from ifes_apt_tc_data_modeling.nexus.nx_ion import NxIon
from ifes_apt_tc_data_modeling.utils.molecular_ions import MolecularIonBuilder
from ifes_apt_tc_data_modeling.utils.utils import MAX_NUMBER_OF_ATOMS_PER_ION
from ifes_apt_tc_data_modeling.utils.nist_isotope_data import isotopes
from ase.data import atomic_numbers, atomic_masses, chemical_symbols

Utility function for exporting (molecular) ions into a simple HDF5 file.

In [ ]:
def simple_hfive_file(fpw, idx, mion):
    """Write specific content in existent and opened HDF5 file pointed to by fpw with write access."""
    trg = f"/entry1/ion{idx}"
    grp = fpw.create_group(trg)
    grp.attrs["NX_class"] = "NXion"
    dst = fpw.create_dataset(f"{trg}/comment", data=ion.comment.typed_value)
    dst = fpw.create_dataset(f"{trg}/color", data=ion.color.typed_value)
    # dst = fpw.create_dataset(f"{trg}/volume", dtype=np.float32, data=0.)
    # dst.attrs["unit"] = "nm^3"
    dst = fpw.create_dataset(f"{trg}/isotope_vector", dtype=np.uint16,
                             data=ion.isotope_vector.typed_value, 
                             chunks=True, compression="gzip", compression_opts=1)
    dst = fpw.create_dataset(f"{trg}/nuclid_list", dtype=np.uint16,
                             data=ion.nuclid_list.typed_value,
                             chunks=True, compression="gzip", compression_opts=1)
    dst = fpw.create_dataset(f"{trg}/charge_state", dtype=np.int8,
                             data=ion.charge_state.typed_value)
    dst = fpw.create_dataset(f"{trg}/name", data=ion.name.typed_value)
    dst = fpw.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                             data=ion.ranges.typed_value)
    dst.attrs["unit"] = "Da"
    subgrpnm = f"{trg}/charge_state_model"
    subgrp = fpw.create_group(subgrpnm)
    subgrp.attrs["NX_class"] = "NXprocess"
    dst = fpw.create_dataset(f"{subgrpnm}/min_abundance", dtype=np.float64,
                             data=ion.charge_state_model["min_abundance"])
    dst = fpw.create_dataset(f"{subgrpnm}/min_abundance_product", dtype=np.float64,
                             data=ion.charge_state_model["min_abundance_product"])
    dst = fpw.create_dataset(f"{subgrpnm}/min_half_life", dtype=np.float64,
                             data=ion.charge_state_model["min_half_life"])
    dst.attrs["unit"] = "s"
    dst = fpw.create_dataset(f"{subgrpnm}/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                             data=ion.charge_state_model["sacrifice_isotopic_uniqueness"])
    dst = fpw.create_dataset(f"{subgrpnm}/isotope_matrix", dtype=np.uint16,
                             data=ion.charge_state_model["isotope_matrix"],
                             chunks=True, compression="gzip", compression_opts=1)
    dst = fpw.create_dataset(f"{subgrpnm}/charge_state_vector", dtype=np.int8,
                             data=ion.charge_state_model["charge_state_vector"])
    dst = fpw.create_dataset(f"{subgrpnm}/mass_vector", dtype=np.float64,
                             data=ion.charge_state_model["mass_vector"])
    dst.attrs["unit"] = "Da"
    dst = fpw.create_dataset(f"{subgrpnm}/nat_abun_prod_vector", dtype=np.float64,
                             data=ion.charge_state_model["nat_abun_prod_vector"])
    dst = fpw.create_dataset(f"{subgrpnm}/min_half_life_vector", dtype=np.float64,
                             data=ion.charge_state_model["min_half_life_vector"])
    dst.attrs["unit"] = "s"

***

## GPM/Rouen ENV system files

<div class="alert alert-block alert-danger">An ENV file has to be formatted using the UTF8 character encoding for this reader to function!</div>

In [ ]:
prefix = f"{os.getcwd()}/data/env/examples_without_provenance"
fnm = ["ErMnO.env"]

In [ ]:
from ifes_apt_tc_data_modeling.env.env_reader import ReadEnvFileFormat

In [ ]:
env = ReadEnvFileFormat(f"{prefix}/{fnm[0]}")
for m_ion in env.env["molecular_ions"]:
    m_ion.report()

## FAU/Erlangen pyccapt control/calibration/ranging HDF5 files

In [ ]:
# eventually change needed point to the location of the files to analyze
prefix = f"{os.getcwd()}/data/env/examples_without_provenance"
fnm = ["1748_Nov-14-2023_13-31_Al.h5", "1748_Al.h5", "1748_Al_range_.h5"]
# df = pd.read_hdf(f"{prefix}/{fnm[1]}")
# H5Web(f"{prefix}/{fnm[1]}")

In [ ]:
from ifes_apt_tc_data_modeling.pyccapt.pyccapt_reader import ReadPyccaptControlFileFormat, ReadPyccaptCalibrationFileFormat, ReadPyccaptRangingFileFormat

In [ ]:
# the actual measurement (comparable to LEAP's STR/RRAW/RHIT/HITS)
pyc_m = ReadPyccaptControlFileFormat(f"{prefix}/{fnm[0]}")

# the calibrations (voltage, bowl, m/q, comparable to LEAP's ROOT and POS/EPOS)
pyc_c = ReadPyccaptCalibrationFileFormat(f"{prefix}/{fnm[1]}")
xyz = pyc_c.get_reconstructed_positions()
print(xyz.typed_value)
m_q = pyc_c.get_mass_to_charge_state_ratio()
print(m_q.typed_value)

# the ranging definitions (comparable to RNG/RRNG)
pyc_r = ReadPyccaptRangingFileFormat(f"{prefix}/{fnm[2]}")

## ATO

In [ ]:
prefix = f"{os.getcwd()}/data/ato/examples_without_provenance"
fnm = ["Si.epos.v3.ATO", "Si.epos.v5.ATO"]
for fpath in fnm:
    print(os.path.getsize(f"{prefix}/{fpath}"))

In [ ]:
from ifes_apt_tc_data_modeling.ato.ato_reader import ReadAtoFileFormat

In [ ]:
for fpath in fnm:
    print(f"{prefix}/{fpath}")
    ato = ReadAtoFileFormat(f"{prefix}/{fpath}")
    
    xyz = ato.get_reconstructed_positions()
    print(xyz.typed_value)
    m_q = ato.get_mass_to_charge_state_ratio()
    print(m_q.typed_value)

## Matlab FIG

<div class="alert alert-block alert-danger">A Matlab FIG has to be first processed using the matlab_fig_to_txt.m script for this reader to function!</div>

In [ ]:
prefix = f"{os.getcwd()}/data/fig/examples_without_provenance/ger_erlangen_felfer"
fnm = ["R56_01769.rng.fig.txt"]

In [ ]:
from ifes_apt_tc_data_modeling.fig.fig_reader import ReadFigTxtFileFormat

In [ ]:
for fpath in fnm:
    if fpath.lower().endswith(".fig.txt"):
        print(fpath)
        figtxt = ReadFigTxtFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in figtxt.fig["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

Display generated HDF5 file using H5Web.

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## RRNG

In [ ]:
prefix = f"{os.getcwd()}/data/rrng/examples_without_provenance/ger_duesseldorf_kuehbach"
fnm = ["R31_06365-v02.rrng"]

In [ ]:
from ifes_apt_tc_data_modeling.rrng.rrng_reader import ReadRrngFileFormat

In [ ]:
for fpath in fnm:
    if fpath.lower().endswith(".rrng"):
        print(fpath)
        rrng = ReadRrngFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{prefix}/{fpath}.nxs", "w") as h5w:
            h5w.create_group("/entry1")
            idx = 1
            for ion in rrng.rrng["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## RNG

In [ ]:
prefix = f"{os.getcwd()}/data/rng/examples_without_provenance/ger_duesseldorf_kuehbach"
fnm = ["SeHoKim_R5076_44076_v02.rng"]

In [ ]:
from ifes_apt_tc_data_modeling.rng.rng_reader import ReadRngFileFormat

In [ ]:
for fpath in fnm:
    if fpath.lower().endswith(".rng"):
        print(fpath)
        rng = ReadRngFileFormat(f"{prefix}/{fpath}")
        with h5py.File(f"{directory}/{filename}.nxs", "w") as h5w:
            h5w.create_group("/entry")
            idx = 1
            for ion in rng.rng["molecular_ions"]:
                simple_hfive_file(h5w, idx, ion)
                idx += 1

In [ ]:
H5Web(f"{prefix}/{fpath}.nxs")

## POS

In [ ]:
prefix = f"{os.getcwd()}/data/pos/examples_without_provenance"
fnm = ["R31_06365-v02.pos"]

In [ ]:
from ifes_apt_tc_data_modeling.pos.pos_reader import ReadPosFileFormat

In [ ]:
for fpath in fnm:
    pos = ReadPosFileFormat(f"{prefix}/{fpath}")
    print(pos.filename)
    print(pos.filesize)
    xyz = pos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = pos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))

## ePOS

In [ ]:
prefix = f"{os.getcwd()}/data/epos/examples_without_provenance"
fnm = ["R18_58152-v02.epos"]

In [ ]:
from ifes_apt_tc_data_modeling.epos.epos_reader import ReadEposFileFormat

In [ ]:
for fpath in fnm:
    epos = ReadEposFileFormat(f"{prefix}/{fpath}")
    print(epos.filename)
    print(epos.filesize)
    xyz = epos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = epos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))
    raw_tof = epos.get_raw_time_of_flight()
    print(np.shape(raw_tof.typed_value))
    st_v = epos.get_standing_voltage()
    print(np.shape(st_v.typed_value))
    p_v = epos.get_pulse_voltage()
    print(np.shape(p_v.typed_value))
    det_xy = epos.get_hit_positions()
    print(np.shape(det_xy.typed_value))
    n_p = epos.get_number_of_pulses()
    print(np.shape(n_p.typed_value))
    h_p = epos.get_ions_per_pulse()
    print(np.shape(h_p.typed_value))

## APT

In [ ]:
prefix = f"{os.getcwd()}/data/apt/examples_without_provenance"
fnm = ["70_50_50.apt"]

In [ ]:
from ifes_apt_tc_data_modeling.apt.apt6_reader import ReadAptFileFormat

In [ ]:
for fpath in fnm:
    apt = ReadAptFileFormat(f"{prefix}/{fpath}")
    print(apt.get_metadata_table())
    xyz = apt.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = apt.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))

## Low-level stuff testing the library

In [ ]:
# ivec = create_isotope_vector(['K-40', 'Xe-126'])  # , 'U', 'H'])
# ivec = create_isotope_vector(['Tc', 'H', 'H'])  # 'C'])
# ivec = create_isotope_vector(['B', 'B', 'B', 'N', 'N', 'Ti'])  # 'N', 'N'])
# ivec = create_isotope_vector([])
ivec = create_isotope_vector(['O', 'O', 'O', 'O'])
ivec = create_isotope_vector(['Fe', 'Fe', 'O', 'O', 'O', 'O', 'O'])
# ivec = create_isotope_vector(['X'])
print(ivec)
# print(isotope_vector_to_dict_keyword(ivec))
# print(isotope_vector_to_human_readable_name(ivec, +4))

## Take the information available in every range file.

In [ ]:
ion = NxIon(isotope_vector=ivec, charge=0)
# ion.add_range((np.floor(atomic_masses[43]) - 2.)*0.5, (np.floor(atomic_masses[43]) + 4.)*0.5)
# ion.add_range(20.2840, 20.3680)
# ion.add_range(54.9540, 54.9630)
# ion.add_range(32.0050,  32.1370)
# ion.add_range(32.4280,  32.4910)
# ion.add_range(32.9600,  33.0640)
# ion.add_range(33.9720,  34.0380)
# ion.add_range(175.8060, 176.2460)
ion.add_range(95.6540, 96.1780)
# ion.add_range(10.5, 13.5)
# ion.add_range(13.0, 13.3)  # should not be prevented because this is equivalent to ion.add_range(10.0, 13.3)
ion.report()

In [ ]:
mion = MolecularIonBuilder(min_abundance=0.,
                           min_abundance_product=0.,
                           min_half_life=np.inf,
                           sacrifice_uniqueness=True,
                           verbose=True)
mion.combinatorics(
    ion.isotope_vector.typed_value,
    ion.ranges.typed_value[0, 0],
    ion.ranges.typed_value[0, 1])

In [ ]:
p, n = hash_to_isotope(1)
print(f"{p}, {n}")

***
Markus Kühbach, 2023/12/29<br>
<br>
<a href="https://www.fairmat-nfdi.eu/fairmat">FAIRmat</a> is a consortium on research data management which is part of the German NFDI.<br>
The project is funded by the Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) – project 460197019.